In [ ]:
import os
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import messagebox, Toplevel, ttk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import joblib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

EXCEL_PATH = "student_data.xlsx"
MODEL_PATH = "student_model.pkl"

# ------------------------- Create Sample Excel -------------------------
def create_sample_excel(path, n=100, seed=42):
    np.random.seed(seed)
    hours = np.random.randint(0, 11, size=n)
    attendance = np.random.randint(40, 101, size=n)
    internal = np.clip((hours * 8) + (attendance * 0.3) + np.random.normal(0, 8, n), 0, 100).astype(int)

    performance = []
    for h, im in zip(hours, internal):
        if im >= 80 and h >= 7:
            performance.append("Excellent")
        elif im >= 60:
            performance.append("Good")
        elif im >= 40:
            performance.append("Average")
        else:
            performance.append("Poor")

    df = pd.DataFrame({
        "Hours_Studied": hours,
        "Attendance": attendance,
        "Internal_Marks": internal,
        "Performance": performance
    })
    df.to_excel(path, index=False, engine='openpyxl')
    return df

def load_or_create_excel(path):
    if os.path.exists(path):
        return pd.read_excel(path, engine='openpyxl')
    else:
        return create_sample_excel(path)

# ------------------------- Train and Save Model -------------------------
def train_and_save_model():
    data = load_or_create_excel(EXCEL_PATH)
    le = LabelEncoder()
    data['Performance_enc'] = le.fit_transform(data['Performance'])
    X = data[['Hours_Studied', 'Attendance', 'Internal_Marks']]
    y = data['Performance_enc']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_train, y_train)
    joblib.dump((model, le), MODEL_PATH)

def predict_performance(hours, attendance, marks):
    if not os.path.exists(MODEL_PATH):
        train_and_save_model()

    model, le = joblib.load(MODEL_PATH)
    sample = [[hours, attendance, marks]]
    pred = model.predict(sample)[0]
    return le.inverse_transform([pred])[0]

# ------------------------- Save Record to Excel -------------------------
def save_student_record(hours, attendance, marks, performance):
    if os.path.exists(EXCEL_PATH):
        df = pd.read_excel(EXCEL_PATH, engine='openpyxl')
    else:
        df = pd.DataFrame(columns=["Hours_Studied", "Attendance", "Internal_Marks", "Performance"])

    new_entry = {
        "Hours_Studied": hours,
        "Attendance": attendance,
        "Internal_Marks": marks,
        "Performance": performance
    }

    df = pd.concat([df, pd.DataFrame([new_entry])], ignore_index=True)
    df.to_excel(EXCEL_PATH, index=False, engine='openpyxl')

# ------------------------- Visualization -------------------------
def show_charts():
    data = load_or_create_excel(EXCEL_PATH)
    fig, axs = plt.subplots(1, 2, figsize=(9, 4))
    fig.suptitle("Student Performance Analysis", fontsize=14, fontweight='bold')

    axs[0].scatter(data["Hours_Studied"], data["Internal_Marks"], c='blue', alpha=0.6)
    axs[0].set_xlabel("Hours Studied")
    axs[0].set_ylabel("Internal Marks")
    axs[0].set_title("Hours Studied vs Internal Marks")

    avg_marks = data.groupby("Performance")["Internal_Marks"].mean()
    axs[1].bar(avg_marks.index, avg_marks.values, color=['green', 'orange', 'blue', 'red'])
    axs[1].set_title("Average Marks by Performance")
    axs[1].set_ylabel("Average Internal Marks")

    plt.tight_layout()

    chart_window = tk.Toplevel()
    chart_window.title("Performance Analysis Charts")
    chart_window.geometry("950x450")

    canvas = FigureCanvasTkAgg(fig, master=chart_window)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

# ------------------------- Admin Panel -------------------------
def open_admin_panel():
    admin_window = Toplevel(root)
    admin_window.title("Admin Dashboard")
    admin_window.geometry("700x400")
    admin_window.configure(bg="#f2f2f2")

    tk.Label(admin_window, text="📊 Admin Dashboard", font=("Arial", 16, "bold"), bg="#f2f2f2", fg="#003366").pack(pady=10)
    tk.Button(admin_window, text="Show Student Records", bg="#004080", fg="white", font=("Arial", 12), command=lambda: show_records(admin_window)).pack(pady=10)
    tk.Button(admin_window, text="Show Charts", bg="#006633", fg="white", font=("Arial", 12), command=show_charts).pack(pady=10)

def show_records(window):
    df = load_or_create_excel(EXCEL_PATH)
    frame = tk.Frame(window)
    frame.pack(fill="both", expand=True, padx=10, pady=10)
    tree = ttk.Treeview(frame, columns=list(df.columns), show="headings")
    for col in df.columns:
        tree.heading(col, text=col)
        tree.column(col, width=120)
    for _, row in df.iterrows():
        tree.insert("", "end", values=list(row))
    tree.pack(fill="both", expand=True)

# ------------------------- User Panel -------------------------
def open_user_panel():
    user_window = Toplevel(root)
    user_window.title("User: Student Performance Predictor")
    user_window.geometry("450x370")
    user_window.configure(bg="#e6f2ff")

    tk.Label(user_window, text="Enter Student Details", font=("Arial", 16, "bold"), bg="#e6f2ff", fg="#003366").pack(pady=10)

    frame = tk.Frame(user_window, bg="#e6f2ff")
    frame.pack(pady=10)

    tk.Label(frame, text="Hours Studied (0–10):", bg="#e6f2ff").grid(row=0, column=0, sticky="w", pady=5)
    entry_hours = tk.Entry(frame)
    entry_hours.grid(row=0, column=1, pady=5)

    tk.Label(frame, text="Attendance (%):", bg="#e6f2ff").grid(row=1, column=0, sticky="w", pady=5)
    entry_attendance = tk.Entry(frame)
    entry_attendance.grid(row=1, column=1, pady=5)

    tk.Label(frame, text="Internal Marks (0–100):", bg="#e6f2ff").grid(row=2, column=0, sticky="w", pady=5)
    entry_marks = tk.Entry(frame)
    entry_marks.grid(row=2, column=1, pady=5)

    def on_predict():
        try:
            hours = float(entry_hours.get())
            attendance = float(entry_attendance.get())
            marks = float(entry_marks.get())

            if not (0 <= hours <= 10 and 0 <= attendance <= 100 and 0 <= marks <= 100):
                messagebox.showerror("Invalid Input", "Please enter values in valid range.")
                return

            result = predict_performance(hours, attendance, marks)
            save_student_record(hours, attendance, marks, result)
            messagebox.showinfo("Prediction Result", f"Predicted Performance: {result}\n✅ Record saved successfully!")

        except ValueError:
            messagebox.showerror("Error", "Please enter valid numeric values!")

    tk.Button(user_window, text="Predict & Save", bg="#004080", fg="white", font=("Arial", 12, "bold"), command=on_predict).pack(pady=10)
    tk.Button(user_window, text="Show Charts 📊", bg="#006633", fg="white", font=("Arial", 12, "bold"), command=show_charts).pack(pady=5)

# ------------------------- Login System -------------------------
def login():
    username = entry_user.get()
    password = entry_pass.get()

    if username == "admin" and password == "admin123":
        messagebox.showinfo("Login Successful", "Welcome, Admin!")
        open_admin_panel()
    elif username == "user" and password == "user123":
        messagebox.showinfo("Login Successful", "Welcome, User!")
        open_user_panel()
    else:
        messagebox.showerror("Access Denied", "Invalid username or password!")

# ------------------------- Root Login Window -------------------------
root = tk.Tk()
root.title("🔐 Student Performance Login System")
root.geometry("400x300")
root.resizable(False, False)
root.configure(bg="#e6f2ff")

tk.Label(root, text="Login to Continue", font=("Arial", 16, "bold"), bg="#e6f2ff", fg="#003366").pack(pady=20)

tk.Label(root, text="Username:", bg="#e6f2ff").pack()
entry_user = tk.Entry(root)
entry_user.pack(pady=5)

tk.Label(root, text="Password:", bg="#e6f2ff").pack()
entry_pass = tk.Entry(root, show="*")
entry_pass.pack(pady=5)

tk.Button(root, text="Login", bg="#004080", fg="white", font=("Arial", 12, "bold"), command=login).pack(pady=20)

root.mainloop()
